## Modules and libraries 

In [ ]:
from cellpose import models, io
from cellpose.io import *
from collections import defaultdict
import glob
import imagej
from jpype import JArray, JInt
import os
import re
import pandas
from pandas import DataFrame
from pathlib import Path
import scyjava
import shutil
import tkinter as tk
from tkinter import filedialog
from PIL import Image

## ImageJ initialization
If under macOS, the interactive mode is not compatible and will yield an error message so it must be removed. 

In [ ]:
scyjava.config.add_option('-Xmx60g') #replace the number by the amount of RAM you want to allocate.
start_dir = os.getcwd()
# open local Fiji installation
ij = imagej.init('/path/to/fiji/Fiji.app/', mode='interactive')
ij.getApp().getInfo(True)
ij.ui().showUI()
os.chdir(start_dir)
ij.getVersion() #This is to make sure ImageJ/Fiji opened properly. In case of error, it could display '2.9.0/inactive' instead of the full version number

In [ ]:
# Here is a few plugins to import
showPolygonRoi = scyjava.jimport('ij.gui.PolygonRoi')
Overlay = scyjava.jimport('ij.gui.Overlay')
Regions = scyjava.jimport('net.imglib2.roi.Regions')
LabelRegions = scyjava.jimport('net.imglib2.roi.labeling.LabelRegions')
ZProjector = scyjava.jimport('ij.plugin.ZProjector')()
ov = Overlay()
rm = ij.RoiManager.getRoiManager()

## Choice of directory
This ask for the parent directory containing of all your subfolders with the image in it

In [ ]:
root = tk.Tk()
root.withdraw()
directory_path = filedialog.askdirectory()

## Creation of individual images with single Z in the T-PMT channel
Depending on you image acquisition parameter or microscope used, the channel number can be adjusted to match your image channel containing the transmitted light images. 
You can change the `wanted_channel` number accordingly. For the `wanted_z`, this has to be adjusted to find the Z in your experiment that can give a good contrasted image that cellpose can succesfully segment. 

In [ ]:
# This is the channel number. The "1" means this is channel one that will be selected.
wanted_channel = 1

# Z-projection method. Can be change to 'sum' or other as needed. 
method = 'max'

# Get a list of all the folders in the directory
folder_list = os.listdir(directory_path)

# Iterate over the folders and process each folder
for folder in folder_list:
    # Get the path to the folder
    folder_path = os.path.join(directory_path, folder)

    # Find all the CZI files in the folder
    file_pattern = os.path.join(folder_path, "*.czi")
    file_list = glob.glob(file_pattern)

    # Iterate through the files and extract the wanted channel
    for file_path in file_list:
        image = ij.io().open(file_path)
        blue_channel = image[:, :, wanted_channel,:]

        # Cellpose segmentation for nuclei worked better with a smoothed nuclear signal, specially with low signal. Smoothing workflow :
        sigma = 1.5  # Adjust the value of sigma as needed
        smoothed_image = ij.op().run("smooth", blue_channel, sigma)
        imp = ij.py.to_imageplus(smoothed_image)

        # Z-projection
        z_projector_result = ZProjector.run(imp, method)
        z_collapsed_image = ij.py.from_java(z_projector_result)
        z_collapsed_dataset = ij.py.to_dataset(z_collapsed_image)  

        # Save as a TIFF image ending with "_blue.tif"
        result_path = os.path.splitext(file_path)[0] + "_blue.tif"
        ij.io().save(z_collapsed_dataset, result_path)
        print(f"Saving image {result_path}.")

## Cellpose segmentation
Depending on your cellpose installation, `gpu=False` can be turned in `true` if cuda has been installed and working. `model_type=''` is to be filled with your model name (see the example below), if the model has been trained or used in the cellpose GUI that you have locally installed. If it's a new Cellpose installation and you want to use your own model, replace `model_type` by `pretrained_model='/full/path/to/model'`. Be sure the path is accurate. 

In [ ]:
folder_list = os.listdir(directory_path)
for folder in folder_list:
    folder_path = os.path.join(directory_path, folder)
    file_pattern = os.path.join(folder_path, "*_blue.tif")
    file_list = glob.glob(file_pattern)
    model = models.CellposeModel(gpu=False, model_type='raw_2')
    for file_path in file_list:
        imgs = io.imread(file_path)
        channels = [[0,0]]
        masks, flows, styles = model.eval(imgs, diameter=None, channels=channels)
        io.save_to_png(imgs, masks, flows, file_path)

## Quantification

### extraction of pstat channel

In [ ]:
method = 'sum'

# Get a list of all the folders in the directory
folder_list = os.listdir(directory_path)

# Iterate over the folders and process each folder
for folder in folder_list:
    # Get the path to the folder
    folder_path = os.path.join(directory_path, folder)

    # Find all the ".czi" files in the folder
    file_pattern = os.path.join(folder_path, "*.czi")
    file_list = glob.glob(file_pattern)

    for file_path in file_list:
        image = ij.io().open(file_path)
        wanted_channel = 2
        green_channel = image[:, :, wanted_channel,:]
        imp = ij.py.to_imageplus(green_channel)
        z_projector_result = ZProjector.run(imp, method)
        z_collapsed_image = ij.py.from_java(z_projector_result)
        z_collapsed_dataset = ij.py.to_dataset(z_collapsed_image)  

    # Save the results, add a suffix "bact" for bacterial channel
        result_path = os.path.splitext(file_path)[0] + "_red.tif"

    # Save as a TIFF image
        ij.io().save(z_collapsed_dataset, result_path)
        print(f"Saving image {result_path}.")

### extraction of ROIs from cellpose and saving them

In [ ]:
folder_list = os.listdir(directory_path)
rm = ij.RoiManager.getRoiManager()

# Iterate over the folders and process each folder
for folder in folder_list:
    # Get the path to the folder
    folder_path = os.path.join(directory_path, folder)

    # Find all the z-projected nuclear files in the folder
    file_pattern = os.path.join(folder_path, "*_blue.tif")
    file_list = glob.glob(file_pattern)
    
    for file_path in file_list:
        image = ij.io().open(file_path)
        f_name = os.path.basename(file_path)
        f_name = os.path.splitext(f_name)[0]
        input_txt = Path(f"{folder_path}/{f_name}_cp_outlines.txt")
        txt_fh = open(input_txt, 'r')
        set_string = f'Set Measurements...'
        measure_string = f'area mean integrated redirect=None decimal=2'
        
        ij.IJ.run(set_string, measure_string)
        roi_stats = defaultdict(list)
        #ij.ui().show(image)
        #to better draw, apply and save ROIs, the image should be displayed:
        imp = ij.py.to_imageplus(image) 
        #Call of .txt cellpose output file and draw ROIs based on segmentation result, and add them in the ROI manager
        for line in txt_fh:
            xy = line.rstrip().split(",")
            xy_coords = [int(element) for element in xy if element not in '']
            x_coords = [int(element) for element in xy[::2] if element not in '']
            y_coords = [int(element) for element in xy[1::2] if element not in '']
            xcoords_jint = JArray(JInt)(x_coords)
            ycoords_jint = JArray(JInt)(y_coords)
            polygon_roi_instance = scyjava.jimport('ij.gui.PolygonRoi')
            roi_instance = scyjava.jimport('ij.gui.Roi')
            imported_polygon = polygon_roi_instance(xcoords_jint, ycoords_jint, len(x_coords), int(roi_instance.POLYGON))
            imp.setRoi(imported_polygon)
            rm.addRoi(imported_polygon)
        ij.py.run_macro("roiManager('Select All');")
        rm.runCommand("Save", f"{folder_path}/" + f"{f_name}.zip")
        rm.runCommand("Delete")

### Voronoi segmentation from nuclei channel

In [ ]:
folder_list = os.listdir(directory_path)
rm = ij.RoiManager.getRoiManager()

# Iterate over the folders and process each folder
for folder in folder_list:
    # Get the path to the folder
    folder_path = os.path.join(directory_path, folder)

    # Find all the z-projected nuclear files in the folder
    file_pattern = os.path.join(folder_path, "*_blue.tif")
    file_list = glob.glob(file_pattern)
    for file_path in file_list:
        image = ij.io().open(file_path)
        f_name = os.path.basename(file_path)
        f_name = os.path.splitext(f_name)[0]
        input_ROI = f"{folder_path}/{f_name}.zip"
        rm.open(input_ROI)
        ij.ui().show(image)
        #to better draw, apply and save ROIs, the image should be displayed:
        imp = ij.py.to_imageplus(image) 
        roi_count = rm.getCount()
        for i in range(roi_count):
            rm.select(i)
            shrinking = ij.IJ.run("Enlarge...", "enlarge=-1 pixel")
            rm.runCommand("update")
        rm.runCommand("Select All")
        rm.runCommand("XOR")
        rm.runCommand("Fill")
        rm.runCommand("Select All")
        rm.runCommand("XOR")
        ij.IJ.run("Clear Outside")
        rm.runCommand("Select All")
        rm.runCommand("Delete")
        ij.py.run_macro('setAutoThreshold("Default dark no-reset");')
        ij.IJ.run("Threshold...")
        ij.py.run_macro('setThreshold(5, 255);')
        ij.py.run_macro('setOption("BlackBackground", true);')
        ij.IJ.run("Convert to Mask", "black")
        
        voronoi = """
run("Set Measurements...", "center redirect=None decimal=1");
	run("Analyze Particles...","size=3-Infinity display clear");
	//Resolution de l'image pwidth et pheight
	getPixelSize(unit, pw, ph, pd);
	//Voronio
	run("Voronoi");
	setThreshold(0, 0,"black & white");
	
	//Wand to ROI Manager
	x=newArray(nResults);
	y=newArray(nResults);
	nbPoints=nResults;
	
	for (i=0; i<nbPoints; i++) {
		x[i]=getResult("XM",i)/pw;
		y[i]=getResult("YM",i)/ph;
	}
	for (i=0; i<nbPoints; i++) {
		doWand(x[i], y[i], 156.0, "Legacy");
		roiManager("Add");
	}

	//Center of mass
	/*
	x=newArray(nResults);
	y=newArray(nResults);
	for (i=0; i<nResults; i++) {
		x[i]=getResult("XM",i)/pw;
		y[i]=getResult("YM",i)/ph;
	}
	*/
	makeSelection("point", x, y);
    selectWindow("Results");
    run("Close");
    close("*");
"""
        run_voronoi = ij.py.run_macro(voronoi)
        ij.py.run_macro("roiManager('Select All');")
        rm.runCommand("Save", f"{folder_path}/" + f"{f_name}_voronoi.zip")
        rm.runCommand("Delete")

### extraction of bacteria channel

In [ ]:
wanted_channel = 0
method = 'sum'

folder_list = os.listdir(directory_path)

for folder in folder_list:
    folder_path = os.path.join(directory_path, folder)
    file_pattern = os.path.join(folder_path, "*.czi")
    file_list = glob.glob(file_pattern)

    for file_path in file_list:
        image = ij.io().open(file_path)
        blue_channel = image[:, :, wanted_channel,:]
        imp = ij.py.to_imageplus(blue_channel)
        z_projector_result = ZProjector.run(imp, method)
        z_collapsed_image = ij.py.from_java(z_projector_result)
        z_collapsed_dataset = ij.py.to_dataset(z_collapsed_image)  

    # Save the results, add a suffix "bact" for bacterial channel
        result_path = os.path.splitext(file_path)[0] + "_bact.tif"
        ij.io().save(z_collapsed_dataset, result_path)
        print(f"Saving image {result_path}.")

### BBQ total fluorescence
Don't forget to adjust the threshold lower value for each new experimment as needed

In [ ]:
folder_list = os.listdir(directory_path)
rm = ij.RoiManager.getRoiManager()

for folder in folder_list:
    folder_path = os.path.join(directory_path, folder)
    file_pattern = os.path.join(folder_path, "*.czi")
    file_list = glob.glob(file_pattern)
    for file_path in file_list:
        basename = os.path.basename(file_path)
        corename = os.path.splitext(basename)[0]
        image_path = f"{folder_path}/{corename}_bact.tif"
        print(image_path)
        image = ij.io().open(image_path)
        ij.ui().show(image)
        imp = ij.py.to_imageplus(image)
        input_ROI = f"{folder_path}/{corename}_blue_voronoi.zip"

        no_background = """
    run("Duplicate...", "duplicate");
    run("Threshold...");
    setThreshold(40, 1000000000000000000000000000000.0000);
    setOption("BlackBackground", true);
    run("Convert to Mask");
    run("Analyze Particles...", "size=10-Infinity pixel add");
    close();
    
    array1 = newArray("0");
    for (i=1;i<roiManager("count");i++){
    array1 = Array.concat(array1,i);
    }
	roiManager("select", array1);
	roiManager("XOR");
    run("Clear Outside");
    roiManager("select", array1);
    roiManager("Delete");
    run("Set Measurements...", "integrated limit redirect=None decimal=2");
    setThreshold(40, 1000000000000000000000000000000.0000, "raw");
    """
        ij.py.run_macro(no_background)
    
        rm.open(f"{input_ROI}")
        Measure = """
    nbArea=roiManager("count")
    for (i=0; i<nbArea; i++) {
		roiManager("Select", i);
		run("Measure");	
	}
    array1 = newArray("0");
    for (i=1;i<roiManager("count");i++){
    array1 = Array.concat(array1,i);
    }
	roiManager("select", array1);
    roiManager("Delete");
    close("*");    
    """
        ij.py.run_macro(Measure)

    measurements = ij.ResultsTable.getResultsTable()
    measurements_table = ij.convert().convert(measurements, scyjava.jimport('org.scijava.table.Table'))
    table = ij.py.from_java(measurements_table)
    results = os.path.basename(folder_path)
    results = os.path.splitext(results)[0]
    output_path = Path(f"{folder_path}/{results}_burden.csv")
    table.to_csv(output_path)

    ij.py.run_macro("""
selectWindow("Results");
run("Clear Results");
""")



### p-stat fluorescence quantification
The end of this part will save a CSV file containing all the values of IntDen and RawIntDen per nuclei in the channel of interest

In [ ]:
for folder in folder_list:
    folder_path = os.path.join(directory_path, folder)
    file_pattern = os.path.join(folder_path, "*.czi")
    file_list = glob.glob(file_pattern)
    for file_path in file_list:
        basename = os.path.basename(file_path)
        corename = os.path.splitext(basename)[0]
        image_path = f"{folder_path}/{corename}_red.tif"
        print(image_path)
        image = ij.io().open(image_path)
        ij.ui().show(image)
        imp = ij.py.to_imageplus(image)
        input_ROI = f"{folder_path}/{corename}_blue.zip"
        rm.open(input_ROI)
        set_string = f'Set Measurements...'
        measure_string = f'area mean integrated redirect=None decimal=2'
        ij.IJ.run(set_string, measure_string)
        Measure = """
    nbArea=roiManager("count")
    for (i=0; i<nbArea; i++) {
		roiManager("Select", i);
		run("Measure");	
	}
    array1 = newArray("0");
    for (i=1;i<roiManager("count");i++){
    array1 = Array.concat(array1,i);
    }
	roiManager("select", array1);
    roiManager("Delete");
    close("*");    
    """
        ij.py.run_macro(Measure)

    
    results = os.path.basename(folder_path)
    results = os.path.splitext(results)[0]
    output_path = f"{folder_path}/{results}_fluo.csv"
    saving = ij.IJ.saveAs("Results", output_path)
    ij.IJ.run("Clear Results")
    ij.py.run_macro('close("*")')